Implmenting section 4 "Hiding the Internal State of the Algorithm" in:

Golle P. (2006) A Private Stable Matching Algorithm. In: Di Crescenzo G., Rubin A. (eds) Financial Cryptography and Data Security. FC 2006. Lecture Notes in Computer Science, vol 4107. Springer, Berlin, Heidelberg. https://doi-org.proxygw.wrlc.org/10.1007/11889663_5

Let $A_1...A_3$ denote 3 men and $B_1...B_3$ denote three women.

Preferences:

$A_1$: $a_1=(0,2,1)$

$A_2$: $a_2=(1,2,0)$

$A_3$: $a_3=(2,1,0)$

---

$B_1$: $b_1=(1,2,0)$

$B_2$: $b_2=(0,2,1)$

$B_3$: $b_3=(0,1,2)$

In [36]:
n = 3

In [77]:
a_1 = [0, 2, 1]
a_2 = [1, 2, 0]
a_3 = [2, 1, 0] 
men_preferences = {"a_1" : a_1, "a_2" : a_2, "a_3" : a_3}

b_1 = [1, 2, 0]
b_2 = [0, 2, 1]
b_3 = [0, 1, 2]
women_preferences = {"b_1" : b_1, "b_2" : b_2, "b_3" : b_3}

In [87]:
def hidden_internal_state(men_prefs, women_prefs):
    # Adding an additional n "fake" men
    for i in range(n):
        fake_man = [0, 1, 2] # The "fake" men prefs are unimportant
        men_prefs["a_{}".format((i + 1) + n)] = fake_man
        
        # Augment women preferences
        for woman in women_prefs:
            women_prefs[woman].append(i + n)
    
    # Computing a stable match
    e_k = [] # Set of engaged men
    f_k = [] # Set of free men
    
    e_k = [man for man in men_preferences][n:] # Initially, all fake men are engaged
    f_k = [man for man in men_preferences][:n] # Initially, all real men are free
    
    
    
    #for k in range(1, (n + 1)):
    #    print(k)
        
    #print(e_k)
    
    #print(men_prefs)
    #print("")
    #print(women_prefs)

In [88]:
hidden_internal_state(men_preferences, women_preferences)

1
2
3
['a_4', 'a_5', 'a_6']
{'a_1': [0, 2, 1], 'a_2': [1, 2, 0], 'a_3': [2, 1, 0], 'a_4': [0, 1, 2], 'a_5': [0, 1, 2], 'a_6': [0, 1, 2]}

{'b_1': [1, 2, 0, 3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5], 'b_2': [0, 2, 1, 3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5], 'b_3': [0, 1, 2, 3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5]}
